QUERING THE KNOWLEDGE BASE

In [1]:
import spacy
from rdflib import Graph, Namespace

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Load RDF Graph
ontology_file = "healthcare_ontology.ttl"
g = Graph()
g.parse(ontology_file, format="ttl")

# Define Namespace
HEALTH = Namespace("http://www.healthcare-ontology.org/")

# Keyword Mapping
keyword_mapping = {
    "treatment": "treats",
    "cure": "treats",
    "therapy": "treats",
    "disease": "Disease",
    "side effect": "hasSideEffect",
    "symptom": "hasSymptom",
    "contraindication": "hasContraindication",
    "avoid": "hasContraindication",
    "drug": "Drug",
    "patient": "Patient"
}

# Function to process user query
def convert_nl_to_sparql(user_query):
    doc = nlp(user_query.lower())  # Process text with NLP model

    subject, relation, obj = None, None, None

    for token in doc:
        for keyword, ontology_term in keyword_mapping.items():
            if keyword in token.text:
                if relation is None:
                    relation = ontology_term
                else:
                    obj = ontology_term

    for ent in doc.ents:
        if subject is None:
            subject = ent.text.capitalize()  # Ensure capitalization matches ontology

    if relation and subject:
        sparql_query = f"""
        PREFIX ex: <http://www.healthcare-ontology.org/>
        SELECT ?result WHERE {{
            ?result ex:{relation} ex:{subject} .
        }}
        """
        return sparql_query
    else:
        return None

# Function to execute SPARQL query
def run_sparql_query(sparql_query):
    if sparql_query:
        results = g.query(sparql_query)
        return [str(row[0]) for row in results]
    else:
        return ["No relevant data found."]

# Example User Query
user_input = "What is the treatment for diabetes?"
sparql_query = convert_nl_to_sparql(user_input)

print("\n🔍 SPARQL Query Generated:\n", sparql_query)
print("\n✅ Query Results:")
print(run_sparql_query(sparql_query))


ModuleNotFoundError: No module named 'spacy'